This works with labeling new piece of data by looking at distance from other training-datas data points and finding k nearest relevant points. Then finding average label associated with them and then assigns that label to new point.

## Car Evaluation Data Set

Attribute Information:

Class Values: unacc, acc, good, vgood

Attributes:

buying: vhigh, high, med, low.

maint: vhigh, high, med, low. 

doors: 2, 3, 4, 5more. 

persons: 2, 4, more. 

lug_boot: small, med, big.

safety: low, med, high.

In [1]:
import numpy as np # numbers
import pandas as pd #data
import matplotlib.pyplot as plt #graphs

In [2]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data', header= None)
data

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [3]:
data[0][3]

'vhigh'

In [4]:
#Rearranges data into numbers but efficiently

# changing buying: vhigh, high, med, low. Into buying: 4,3,2,1 #collumn 0
# changing maint: vhigh, high, med, low. INTO 4,3,2,1 #collumn 1
#doors: 2, 3, 4, 5more. Change 5more to 5. #collumn 2
# persons: 2, 4, more. change more to 5 #collumn 3
#lug_boot: small, med, big. change to 1, 2,3 #collumn 4
#safety: low, med, high. change to 1, 2,3 #collumn 5


for n in range(6):
    for i in range(len(data[n])):
        if data[n][i]=='5more' or data[n][i]=='more':
            data[n][i]=5
        elif data[n][i]=='vhigh':
            data[n][i]=4
        elif data[n][i]=='high' or data[n][i]=='big':
            data[n][i]=3
        elif data[n][i]=='med':
            data[n][i]=2
        elif data[n][i]=='low' or data[n][i]=='small':
            data[n][i]=1
        else:
            data[n][i]=int(data[n][i])
data

,0,1,2,3,4,5,6
0,4,4,2,2,1,1,unacc
1,4,4,2,2,1,2,unacc
2,4,4,2,2,1,3,unacc
3,4,4,2,2,2,1,unacc
4,4,4,2,2,2,2,unacc
...,...,...,...,...,...,...,...
1723,1,1,5,5,2,2,good
1724,1,1,5,5,2,3,vgood
1725,1,1,5,5,3,1,unacc
1726,1,1,5,5,3,2,good


In [5]:
X = data.iloc[:, :-1].values #all rows and all collumns up until last
y = data.iloc[:, -1].values #all rows but just last collumn

X
#y

array([[4, 4, 2, 2, 1, 1],
       [4, 4, 2, 2, 1, 2],
       [4, 4, 2, 2, 1, 3],
       ...,
       [1, 1, 5, 5, 3, 1],
       [1, 1, 5, 5, 3, 2],
       [1, 1, 5, 5, 3, 3]], dtype=object)

What do I do about distance judging with more than 4 collumns is it still  itemgetter(2)??

This just sorts the already found distances by the 3rd element. 
So dont have to change this
distances.sort(key=operator.itemgetter(2))

It is this line
distance = np.linalg.norm(X[i] - test_point)
that measures the distance

In [6]:
import operator
from pprint import pprint #short for pretty print

def knn(X, y, test_point, k):
    distances = [] #empty list (keeping track of distances and class in a list)
    for i in range(len(X)):
        distance = np.linalg.norm(X[i] - test_point) #keeping track of the distances
        distances.append((y[i], X[i], distance)) #this keeps track of 
        #(label, data point, distance of that data point to that test point)
    
    distances.sort(key=operator.itemgetter(2)) #sort our list of distances by 3rd element element 
    #list of tuples, sorting a list of something more complicated than numbers
    #sorted distance with smallest distance at the top of the list.
    
    #now we want to find the most frequent label,(ie y[i]), in the k nearest neighbours
   
    nn_labels = [] #list to keep track of this
    for nn in distances[:k]: #iterate over first k distances (ie k smallest distances from testpoint)
        nn_labels.append(nn[0]) #append to list the label y[i] for element
        #iteration means in end we get a list of labels of each element in these first k distances
    
    from statistics import mode
    y_hat = mode(nn_labels) #most common element of this list 
    #ie most common label for the the k nearest neighbours
    
    
    print("Test point: "+str(test_point))
    print("Closest K neighbours")
    pprint(distances[:k]) #using pprint to print clearer
    print("Furtherest K neighbours")
    pprint(distances[-k:]) #furthest k distances away, from -k to end of array
    
    return y_hat
    
    

In [7]:
#testing on data point to see if its catagorised right
idx = 5

test_point = X[idx]
test_point_label = y[idx]

X = np.delete(X, idx, axis=0) 
#cardinal sin to train on testing set so delete this x from testing set X
#delete row from x so not training on dataset
X[idx]

array([4, 4, 2, 2, 3, 1], dtype=object)

In [8]:
#test where now 
y_hat = knn(X, y, test_point, 1) #1 nearest neighbours
print("Prediction: "+str(y_hat))
print("Actual: "+str(y[idx]))

Test point: [4 4 2 2 2 3]
Closest K neighbours
[('unacc', array([4, 4, 2, 2, 1, 3], dtype=object), 1.0)]
Furtherest K neighbours
[('vgood', array([1, 1, 5, 5, 3, 1], dtype=object), 6.4031242374328485)]
Prediction: unacc
Actual: unacc


In [9]:
#test where now 
y_hat = knn(X, y, test_point, 3) #3 nearest neighbours
print("Prediction: "+str(y_hat))
print("Actual: "+str(y[idx]))

Test point: [4 4 2 2 2 3]
Closest K neighbours
[('unacc', array([4, 4, 2, 2, 1, 3], dtype=object), 1.0),
 ('unacc', array([4, 4, 2, 2, 2, 2], dtype=object), 1.0),
 ('unacc', array([4, 4, 2, 2, 3, 3], dtype=object), 1.0)]
Furtherest K neighbours
[('good', array([1, 1, 5, 5, 2, 1], dtype=object), 6.324555320336759),
 ('vgood', array([1, 1, 5, 5, 1, 1], dtype=object), 6.4031242374328485),
 ('vgood', array([1, 1, 5, 5, 3, 1], dtype=object), 6.4031242374328485)]
Prediction: unacc
Actual: unacc


In [10]:
#test where now 
y_hat = knn(X, y, test_point, 10) #10 nearest neighbours
print("Prediction: "+str(y_hat))
print("Actual: "+str(y[idx]))

Test point: [4 4 2 2 2 3]
Closest K neighbours
[('unacc', array([4, 4, 2, 2, 1, 3], dtype=object), 1.0),
 ('unacc', array([4, 4, 2, 2, 2, 2], dtype=object), 1.0),
 ('unacc', array([4, 4, 2, 2, 3, 3], dtype=object), 1.0),
 ('unacc', array([4, 4, 3, 2, 2, 3], dtype=object), 1.0),
 ('unacc', array([4, 3, 2, 2, 2, 3], dtype=object), 1.0),
 ('unacc', array([3, 4, 2, 2, 2, 3], dtype=object), 1.0),
 ('unacc', array([4, 4, 2, 2, 1, 2], dtype=object), 1.4142135623730951),
 ('unacc', array([4, 4, 2, 2, 3, 2], dtype=object), 1.4142135623730951),
 ('unacc', array([4, 4, 3, 2, 1, 3], dtype=object), 1.4142135623730951),
 ('unacc', array([4, 4, 3, 2, 2, 2], dtype=object), 1.4142135623730951)]
Furtherest K neighbours
[('vgood', array([1, 1, 5, 4, 3, 1], dtype=object), 6.0),
 ('good', array([1, 1, 5, 5, 2, 3], dtype=object), 6.0),
 ('acc', array([1, 1, 5, 5, 1, 3], dtype=object), 6.082762530298219),
 ('unacc', array([1, 1, 5, 5, 2, 2], dtype=object), 6.082762530298219),
 ('good', array([1, 1, 5, 5, 3, 